# GroupfMRI6a - Use motor rule regions to predict Motor response regions
#### Using ActFlow

#### Takuya Ito
#### 03/01/2019

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
import scipy.stats as stats
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import seaborn as sns
import h5py
import tools_group
import nibabel as nib
sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


/home/tito/miniconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 1.0 Define functions for motor response decodings

#### Load in FC mappings

In [5]:
# import GroupfMRI5a_ComputePCA_FC as tmp
# tmp = reload(tmp)
# inputtypes = ['ori','color','pitch','constant']
# for inputtype in inputtypes:
#     print 'Computing group fc for inputtype', inputtype
#     tmp.computeGroupFCMappings(inputtype)

In [4]:
tools_group = reload(tools_group)
fc_input2sr, fc_sr2motorrule, fc_motorrule2motorresp = tools_group.loadGroupActFlowFC('ori')

## 1.01 Load data for RH and LH responses

In [7]:
tools_group = reload(tools_group)
inputtype = 'ori'
data_task_rh = np.zeros((len(glasser2),2,len(subjNums)))
data_task_lh = np.zeros((len(glasser2),2,len(subjNums)))
actflow_rh = np.zeros(data_task_rh.shape)
actflow_lh = np.zeros(data_task_lh.shape)

# Create arrays for visualization
layerByLayerActivityPredictions_rmid = np.zeros((len(glasser2),7))
layerByLayerActivityPredictions_rind = np.zeros((len(glasser2),7))

# Identify target indices
targetdir = '/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/'
motor_resp_regions_LH = np.loadtxt(targetdir + 'MotorResponseRegions_LH.csv',delimiter=',')
motor_resp_regions_RH = np.loadtxt(targetdir + 'MotorResponseRegions_RH.csv',delimiter=',')
targetROIs = np.hstack((motor_resp_regions_LH,motor_resp_regions_RH))
        
target_ind = []
for roi in targetROIs:
    roi_ind = np.where(glasser2==roi+1)[0]
    target_ind.extend(roi_ind)
target_ind = np.asarray(target_ind)

scount = 0
for subj in subjNums:
    print 'Running compositional analysis on subject', scount+1, '/', len(subjNums)
    tmp1, tmp2, tmp3, input_ind, sr_ind, motorrule_ind = tools_group.compositionalActFlowWithoutFC(subj, inputtype)
    data_input = tmp1
    data_sr = tmp2
    data_motorrule = tmp3
    
    data_task_rh[:,:,scount] = tools_group.loadMotorResponses(subj,hand='Right')
    data_task_lh[:,:,scount] = tools_group.loadMotorResponses(subj,hand='Left')
    
    #####################
    # Run composition activity flow
    
    ## Run for RH
    #### Input to SR regions
    # first identify non-overlapping indices
    unique_input_ind = np.where(np.in1d(input_ind,sr_ind)==False)[0]
    actflow_sr_rmid = np.dot(data_input[unique_input_ind,0],fc_input2sr) # rmid - vert vert stim
    actflow_sr_rind = np.dot(data_input[unique_input_ind,3],fc_input2sr) # rind - horiz horiz stim
    # Construct composition sr representations
    unique_sr_ind = np.where(np.in1d(sr_ind,motorrule_ind)==False)[0]
    sr_composition_rmid = np.multiply(actflow_sr_rmid, data_sr[:,0]) #BOTH
    sr_composition_rind = np.multiply(actflow_sr_rind, data_sr[:,0]) # BOTH
    actflow_motorrule_rmid = np.dot(sr_composition_rmid[unique_sr_ind],fc_sr2motorrule)
    actflow_motorrule_rind = np.dot(sr_composition_rind[unique_sr_ind],fc_sr2motorrule)
    # Construct motor rule compositions
    unique_motor_ind = np.where(np.in1d(motorrule_ind,target_ind)==False)[0]
    motorrule_composition_rmid = np.multiply(actflow_motorrule_rmid, data_motorrule[:,2]) # This is rmid
    motorrule_composition_rind = np.multiply(actflow_motorrule_rind, data_motorrule[:,2]) # This is rmid
    actflow_rh[target_ind,0,scount] = np.dot(motorrule_composition_rmid[unique_motor_ind],fc_motorrule2motorresp)
    actflow_rh[target_ind,1,scount] = np.dot(motorrule_composition_rind[unique_motor_ind],fc_motorrule2motorresp)
    
    ## Run for LH
    #### Input to SR regions
    # first identify non-overlapping indices
    unique_input_ind = np.where(np.in1d(input_ind,sr_ind)==False)[0]
    actflow_sr_lmid = np.dot(data_input[unique_input_ind,0],fc_input2sr) # lmid - vert vert stim
    actflow_sr_lind = np.dot(data_input[unique_input_ind,3],fc_input2sr) # lind - horiz horiz stim
    # Construct composition sr representations
    unique_sr_ind = np.where(np.in1d(sr_ind,motorrule_ind)==False)[0]
    sr_composition_lmid = np.multiply(actflow_sr_lmid, data_sr[:,0]) #BOTH
    sr_composition_lind = np.multiply(actflow_sr_lind, data_sr[:,0]) # BOTH
    actflow_motorrule_lmid = np.dot(sr_composition_lmid[unique_sr_ind],fc_sr2motorrule)
    actflow_motorrule_lind = np.dot(sr_composition_lind[unique_sr_ind],fc_sr2motorrule)
    # Construct motor rule compositions
    unique_motor_ind = np.where(np.in1d(motorrule_ind,target_ind)==False)[0]
    motorrule_composition_lmid = np.multiply(actflow_motorrule_lmid, data_motorrule[:,0])
    motorrule_composition_lind = np.multiply(actflow_motorrule_lind, data_motorrule[:,0])
    actflow_lh[target_ind,0,scount] = np.dot(motorrule_composition_lmid[unique_motor_ind],fc_motorrule2motorresp)
    actflow_lh[target_ind,1,scount] = np.dot(motorrule_composition_lind[unique_motor_ind],fc_motorrule2motorresp)
    #####################
    
    scount += 1
    

del fc_input2sr, fc_motorrule2motorresp, fc_sr2motorrule

Running compositional analysis on subject 1 / 96


tools_group.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,0] = data[:,2]
tools_group.py:97: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,1] = data[:,3]
tools_group.py:99: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,0] = data[:,0] #lmid
tools_group.py:100: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,1] = data[:,1] #lind
/home/tito/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:54: ComplexWarning: Casting complex values to real discards the imaginary part
/home/tito/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:55: ComplexWarning: Casting complex values to real discards the imaginary part
/home/tito/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:73: ComplexWarning: Casting complex values to real discards the imaginary part
/home/tito/miniconda2/lib/python2.7/site-packages/ipyk

Running compositional analysis on subject 2 / 96
Running compositional analysis on subject 3 / 96
Running compositional analysis on subject 4 / 96
Running compositional analysis on subject 5 / 96
Running compositional analysis on subject 6 / 96
Running compositional analysis on subject 7 / 96
Running compositional analysis on subject 8 / 96
Running compositional analysis on subject 9 / 96
Running compositional analysis on subject 10 / 96
Running compositional analysis on subject 11 / 96
Running compositional analysis on subject 12 / 96
Running compositional analysis on subject 13 / 96
Running compositional analysis on subject 14 / 96
Running compositional analysis on subject 15 / 96
Running compositional analysis on subject 16 / 96
Running compositional analysis on subject 17 / 96
Running compositional analysis on subject 18 / 96
Running compositional analysis on subject 19 / 96
Running compositional analysis on subject 20 / 96
Running compositional analysis on subject 21 / 96
Running 

#### Construct actflow predictions for 
* BOTH, VERTICAL, LEFT MIDDLE
* STIM: VERTICAL X VERTICAL, HORIZONTAL X HORIZONTAL

## 1.1 Run across subject decoding on right-hand motor responses

In [8]:
nproc = 20
nResponses = 2
ncvs = 1

rois = [8,9]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi)[0])

# realdata = stats.zscore(data_task_rh[target_ind,:,:],axis=0).copy()
# flowata = stats.zscore(actflow_data[target_ind,:,:],axis=0).copy()
# realdata = data_task_rh[target_ind,:,:].copy()
# flowdata = actflow_rh[target_ind,:,:].copy()
realdata = data_task_rh[roi_ind,:,:].copy()
flowdata = actflow_rh[roi_ind,:,:].copy()


distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:] = tools_group.actflowDecodings(realdata,
                                                          flowdata,
                                                          ncvs=ncvs, nproc=nproc)

# distances_baseline_rh[0,:] = tools_group.actflowDecodings(realdata,
#                                                           data_composition_rh[target_ind,:,:],
#                                                           ncvs=ncvs, nproc=nproc)
    

## 1.2 Compute statistics

In [9]:
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print 'Activity flow accuracy =', statistics_rh[0,0]
print 'p =', statistics_rh[0,1]



Activity flow accuracy = 0.395833333333
p = 0.997622381257


# 2.0 Run across subject decoding on left-hand motor responses

In [10]:
tools_group = reload(tools_group)

In [11]:
nproc = 20
ncvs = 1

rois = [188,189]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi)[0])
    
# realdata = stats.zscore(data_task_lh[target_ind,:,:],axis=0).copy()
# flowdata = stats.zscore(actflow_lh[target_ind,:,:],axis=0).copy()
# realdata = data_task_lh[target_ind,:,:].copy()
# flowdata = actflow_lh[target_ind,:,:].copy()
realdata = data_task_lh[roi_ind,:,:].copy()
flowdata = actflow_lh[roi_ind,:,:].copy()


distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:] = tools_group.actflowDecodings(realdata,
                                                          flowdata,
                                                          ncvs=ncvs, nproc=nproc)

# distances_baseline_lh[0,:] = tools_group.actflowDecodings(realdata,
#                                                           data_composition_lh[target_ind,:,:],
#                                                           ncvs=ncvs, nproc=nproc)
    

## 1.2 Compute statistics

In [12]:
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print 'Activity flow accuracy =', statistics_lh[0,0]
print 'p =', statistics_lh[0,1]



Activity flow accuracy = 0.552083333333
p = 0.0850739297982
